# Tuning a LR Model

copied from `xgb_fitting.ipynb`

The goal of this notebook is to train and evaluate an LR model, comparing it's performance on a holdout set against other types of models (LR,LDA, XGBoost). 

To ensure reproducibility and consistent evaluation across models, all datasets were **pre-split into cross-val data and holdout data** as below:

| Split type           | CV training file     | Holdout file              | Description                              |
| -------------------- | -------------------- | ------------------------- | ---------------------------------------- |
| **Random**           | `apps_cv_random.csv` | `apps_holdout_random.csv` | Simple random sampling                   |
| **Stratified**       | `apps_cv_strat.csv`  | `apps_holdout_strat.csv`  | Stratified by `TARGET`                   |
| **Multi-Stratified** | `apps_cv_multi.csv`  | `apps_holdout_multi.csv`  | Stratified by `TARGET` + `CODE_GENDER_M` |

Each dataset for cross-validation (`apps_cv_*.csv`) also contains a column, `fold`, with pre-assigned folds from 1-5 using the corresponding splitting method to ensure consistent evaluation. Therefore, no additional splitting is needed inside this notebook -- can simply loop through assigned folds for cross-validation.


## Evaluation Functions

#### Metric calculators:

Copied from `cross_val.ipynb`

In [1]:
# METRICS 

def classification_metrics(y_true, y_pred):
    """
    Computes confusion matrix + accuracy, precision, recall, F1, and balanced accuracy.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Confusion matrix components
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    # Metrics
    acc  = (tp + tn) / max((tp + tn + fp + fn), 1)
    prec = tp / max((tp + fp), 1)
    rec  = tp / max((tp + fn), 1)
    f1   = (2 * prec * rec / max((prec + rec), 1e-12)) if (prec + rec) > 0 else 0.0

    # Specificity (True Negative Rate)
    spec = tn / max((tn + fp), 1)

    # Balanced accuracy
    bal_acc = 0.5 * (rec + spec)

    metrics = {
        "n": len(y_true),
        "tp": tp, "tn": tn, "fp": fp, "fn": fn,
        "acc": acc, "bal_acc": bal_acc, "prec": prec, "rec": rec, "spec": spec,
        "f1": f1
    }
    return metrics

def roc_auc_from_probs(y_true, y_prob):
    
    desc_sort_indices = np.argsort(-y_prob)
    y_true = np.array(y_true)[desc_sort_indices]
    y_prob = np.array(y_prob)[desc_sort_indices]
    pos = np.sum(y_true == 1)
    neg = np.sum(y_true == 0)

    # running totals for TPR/FPR
    tpr = [0.0]
    fpr = [0.0]
    tp = fp = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            tp += 1
        else:
            fp += 1
        tpr.append(tp / pos)
        fpr.append(fp / neg)

    # get auc
    auc = np.trapezoid(tpr, fpr)
    return auc

In [2]:
import pandas as pd 
import numpy as np 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [3]:
def cv_lr(data, feature_cols, target_col, params=None):
    if params is None:
        # params = {
        #     "penalty": "l2",
        #     "C": 1.0,
        #     "solver": "lbfgs",
        #     "class_weight": "balanced",
        #     "max_iter": 500
        # }
        params = {}

    fold_metrics = []
    for f in sorted(data.fold.unique()):

        # split data into train and test folds
        train = data[data.fold != f]
        test  = data[data.fold == f]
        X_train, y_train = train[feature_cols], train[target_col]
        X_test,  y_test  = test[feature_cols],  test[target_col]

        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('pca', PCA(0.95)),
            ('lr', LogisticRegression(**params))
        ])
        pipeline.fit(X_train, y_train)

        y_pred       = pipeline.predict(X_test)
        y_prob       = pipeline.predict_proba(X_test)[:, 1]
        y_train_prob = pipeline.predict_proba(X_train)[:, 1]

        m = classification_metrics(y_test, y_pred)
        m["roc_auc"]       = roc_auc_from_probs(y_test, y_prob)
        m["train_roc_auc"] = roc_auc_from_probs(y_train, y_train_prob)
        m["fold"] = int(f)

        fold_metrics.append(m)

    return pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)

## setup

In [4]:
apps_cv_strat = pd.read_csv("data/apps_cv_strat.csv")
apps_holdout_strat = pd.read_csv("data/apps_holdout_strat.csv")
target_col = 'TARGET'
feature_cols = [col for col in apps_cv_strat.columns if col not in 
                [target_col, 'SK_ID_CURR', 'fold', 'neighbors_target_mean_500']]

In [11]:
apps_cv_strat.drop(columns=["CODE_GENDER_XNA","CODE_GENDER_M","AGE_INT"])

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,prev_last_decision_status_internal_Refused,prev_last_decision_status_internal_Unused offer,most_recent_loan_active_external_group_Closed,most_recent_loan_active_external_group_Problematic,most_recent_loan_type_external_group_Business/Other,most_recent_loan_type_external_group_Consumer/Personal,most_recent_loan_type_external_group_Mortgage/Real estate,prev_last_decision_status_internal_group_Canceled,prev_last_decision_status_internal_group_Refused,fold
0,384635,0,1,135000.0,1078200.0,34780.5,900000.0,0.010966,-14899,-4019,...,False,False,False,False,False,True,False,False,False,5
1,384639,0,0,157500.0,900000.0,26316.0,900000.0,0.035792,-20671,365243,...,False,False,False,False,False,True,False,True,False,3
2,384641,0,0,180000.0,490495.5,27387.0,454500.0,0.018029,-20168,-1161,...,False,False,False,False,False,True,False,False,False,4
3,384642,0,0,135000.0,508495.5,38146.5,454500.0,0.072508,-14048,-3569,...,False,False,True,False,False,True,False,False,False,3
4,384645,1,0,180000.0,225000.0,20637.0,225000.0,0.010006,-11025,-4194,...,False,False,False,False,False,True,False,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245772,240755,0,0,112500.0,219870.0,9814.5,157500.0,0.010966,-19845,-706,...,False,False,False,False,False,True,False,False,False,3
245773,240978,0,0,58500.0,970380.0,28503.0,810000.0,0.002134,-22619,365243,...,False,False,True,False,False,True,False,False,False,2
245774,241868,0,1,157500.0,675000.0,32602.5,675000.0,0.035792,-11050,-443,...,False,False,False,False,False,True,False,False,False,1
245775,242417,1,0,135000.0,547272.0,43366.5,495000.0,0.020246,-19347,-4798,...,False,False,False,False,False,True,False,True,False,3


In [12]:
results_lr = cv_lr(apps_cv_strat,feature_cols,target_col)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3

In [13]:
results_lr

,n,tp,tn,fp,fn,acc,bal_acc,prec,rec,spec,f1,roc_auc,train_roc_auc,fold
0,49156,68,45128,59,3901,0.919440,0.507914,0.535433,0.017133,0.998694,0.033203,0.757047,0.756912,1
1,49156,74,45117,70,3895,0.919338,0.508548,0.513889,0.018644,0.998451,0.035983,0.756781,0.756990,2
2,49156,80,45121,66,3889,0.919542,0.509348,0.547945,0.020156,0.998539,0.038882,0.756078,0.757110,3
3,49155,73,45117,69,3896,0.919337,0.508433,0.514085,0.018393,0.998473,0.035514,0.755285,0.757154,4
4,49154,76,45132,54,3892,0.919722,0.508979,0.584615,0.019153,0.998805,0.037091,0.752719,0.758220,5


## todo: grid search/ hyperparam tuning

In [32]:
from sklearn.model_selection import ParameterGrid
import pandas as pd
import numpy as np

def grid_search_lr(data, feature_cols, target_col, param_grid):
    rows = []
    best = {"mean_auc": -np.inf, "params": None}

    for i, params in enumerate(ParameterGrid(param_grid), start=1):
        cv = cv_lr(data, feature_cols, target_col, params=params)  
        mean_auc = cv["roc_auc"].mean()
        std_auc  = cv["roc_auc"].std()

        rows.append({
            **params,
            "mean_auc": mean_auc,
            "std_auc": std_auc,
            "mean_recall": cv["rec"].mean(),
            "mean_f1": cv["f1"].mean(),
            "mean_acc": cv["acc"].mean(),
            "mean_bal_acc": cv["bal_acc"].mean(),
        })

        if mean_auc > best["mean_auc"]:
            best = {"mean_auc": mean_auc, "params": params}

        if i % 5 == 0:
            print(f"{i} combos | best AUC={best['mean_auc']:.4f} | best={best['params']}")

    results = pd.DataFrame(rows).sort_values("mean_auc", ascending=False).reset_index(drop=True)
    return results, best["params"]

In [33]:
param_grid = {
    "penalty": ["l2"],
    "C": [0.01, 0.1, 1, 10, 100],    
    "solver": ["lbfgs", "saga"],    
    "class_weight": ["balanced"],    
    "max_iter": [500]
}

In [ ]:
lr_grid_results, best_lr_params = grid_search_lr(apps_cv_strat, feature_cols, target_col, param_grid)
lr_grid_results.head(), best_lr_params

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3

5 combos | best AUC=0.7572 | best={'C': 1, 'class_weight': 'balanced', 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3

In [ ]:
lr_grid_results

## holdout set

In [11]:
X_train, y_train = apps_cv_strat[feature_cols], apps_cv_strat[target_col]
X_test,  y_test  = apps_holdout_strat[feature_cols], apps_holdout_strat[target_col]

params = {
            "penalty": "l2",
            "C": 1.0,
            "solver": "lbfgs",
            "class_weight": "balanced",
            "max_iter": 500
        }

pipe = Pipeline([
    ("scale", StandardScaler()),
    ("pca", PCA(n_components=0.95)),
    ("lr", LogisticRegression(**params)),
])

pipe.fit(X_train, y_train)
y_pred       = pipe.predict(X_test)
y_prob       = pipe.predict_proba(X_test)[:, 1]
y_train_prob = pipe.predict_proba(X_train)[:, 1]


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3

In [12]:
metrics = classification_metrics(y_test, y_pred)
metrics['roc_auc'] = roc_auc_from_probs(y_test, y_prob)
metrics['train_roc_auc'] = roc_auc_from_probs(y_train, y_train_prob)
metrics = pd.DataFrame([metrics])

In [13]:
metrics

,n,tp,tn,fp,fn,acc,bal_acc,prec,rec,spec,f1,roc_auc,train_roc_auc
0,61443,3419,39481,17002,1541,0.698208,0.694152,0.167426,0.689315,0.698989,0.269414,0.756646,0.75769


## leakage check

In [15]:
shuffled = apps_cv_strat.copy()
shuffled['TARGET'] = np.random.permutation(shuffled['TARGET'].values)
fold_results_shuffled = cv_lr(shuffled, feature_cols, target_col, params)
print("Shuffled mean AUC:", fold_results_shuffled.roc_auc.mean())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_pca.py:604: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3

Shuffled mean AUC: 0.5024842111752665


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: divide by zero encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (1, -1)) @ self.components_.T
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: over